In [2]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# Connecting to OpenAI

In [4]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [5]:
openai = OpenAI()

In [29]:
class WebSite:

    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [30]:
ed = WebSite('https://edwarddonner.com/')

In [35]:
print(ed.title)

Home - Edward Donner


# Types of prompts

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [36]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [42]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [43]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

# Messages

The API from OpenAI expects to receive messages in a particular structure. Many of the other APIs share this structure:

[

    {
        "role": "system", 
        "content": "system message goes here"
    },
    {
        "role": "user", 
        "content": "user message goes here"
    }
    
]


In [53]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [55]:
print(messages_for(ed))

[{'role': 'system', 'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'}, {'role': 'user', 'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a mass

- Time to bring it together - the API for OpenAI is very simple!

In [62]:
def summarize(url):
    website = WebSite(url)
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_for(website)
    )
    return response.choices[0].message.content

In [59]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [60]:
display_summary(url='https://edwarddonner.com/')

# Summary of Home - Edward Donner

Edward Donner's website showcases his interests and professional endeavors related to artificial intelligence and LLMs (Large Language Models). As the co-founder and CTO of Nebula.io, Edward focuses on leveraging AI to help people discover their potential, particularly in the talent management space. His background includes founding the AI startup untapt, which was acquired in 2021.

### Interests and Activities
- Passion for writing code and experimenting with LLMs.
- Enjoys DJing and electronic music production, albeit at an amateur level.
- Engages with the tech community through platforms like Hacker News.

### News and Announcements
- **May 28, 2025**: Launch of courses designed to help individuals become experts and leaders in LLM technology.
- **May 18, 2025**: Announcement of the 2025 AI Executive Briefing.
- **April 21, 2025**: Introduction of the Complete Agentic AI Engineering Course.
- **January 23, 2025**: Announcement of a workshop focused on hands-on experience with agents, including resources for attendees.

For further connection and updates, Edward encourages visitors to reach out through his contact information provided on the site.

In [63]:
display_summary("https://cnn.com")

# CNN Website Summary

CNN provides extensive coverage of breaking news, offering a variety of topics that include US news, world events, politics, business, health, entertainment, sports, science, and climate issues. The site encompasses a range of multimedia content, including articles, videos, and live TV streaming. 

## Current News Highlights
- **US Politics**: A watchdog agency has launched a probe into former Trump prosecutor Jack Smith. Analysis indicates a significant turn for the Trump administration, invoking Orwellian themes.
- **International Affairs**: Israel's Netanyahu has postponed a decision on military action in Gaza, while US envoy Witkoff is visiting aid sites amidst a deepening starvation crisis in the region.
- **Business**: Trump’s tariffs are shifting African nations towards China; drugmakers are investing heavily in US manufacturing, aiming for domestic supply chains.
- **Science and Environment**: Researchers discover a strange 'black goo' containing unknown microorganisms on a ship, and Australia has identified a new supersized species of stick insect.

## Social Issues
- The site discusses various social themes, including the impact of loneliness, the experience of Korean adoptees reuniting with long-lost parents, and controversies surrounding cultural figures such as the NYC shooter case.

This website serves as a comprehensive source for the latest news and insightful analysis on pressing global and domestic issues.